# Team Fortress 2 Economics

This notebook is a work-in-progress study of the Team Fortress 2 economy. It utilizes the `backpack.tf` [price history API](https://backpack.tf/api/pricehistory). The Python code backending the analysis is my own, but massive credit goes to [Fiskie](http://steamcommunity.com/profiles/76561198012598620/), the `backpack.tf` developer who put together the API that made this possible.

Great effort has been made to make this notebook read on a generally accessible level.

### FAQ

**What is Team Fortress 2?**

Team Fortress 2 is a first-person team-based multiplayer shooter video game that was initially released in 2007. The game has recieved continual love and attention from its publisher, Valve, ever since then, which a handful of major content updates released every year and a variety of smaller fixes and additions released throughout. The result is that the game remains viable today, though I personally would say that at this point—going on eight years post-release—its glory days are behind it.

If you are not already familiar with the game, the [Team Fortress 2](https://en.wikipedia.org/wiki/Team_Fortress_2) Wikipedia article is, as always, a good reference point.

**What is the Team Fortress 2 economy? Where did it come from?**

Team Fortress 2 was originally a pay-to-play game: at the time of its 2007 release you had to buy the game disc, or download it from the then still new [Steam](https://en.wikipedia.org/wiki/Steam_%28software%29) platform, in order to play it; once you owned the game you owned the game, and Valve neither had nor produced any further incentive for your monetary investment in the game.

Upon release in 2007, TF2 featured a small number of maps, a few stock weapons, and gratuitous cartoon violence—but little else. Now, fast forward to Septmeber 2010. Successive updates have introduced a small armory of new weapons, the item crafting system, hats, and other such nicities, but it's been three years now, and the game is turning into a money sink for
Valve. The Team Fortress 2 economy was their clever solution: a storefront was introduced allowing players to spend real word money on pixilated cosmetics and player items, and, simultaneously and critically, a trading interface. To encourage trading and valuation of goods they turned to the market: a trading interface was created that allowed people to swap items with one another.

The economy has had its ups and downs and bumps ever since then, but one thing is clear: it made Valve money; a lot of money. No one knows exactly how much money is locked up in the Team Fortress 2 economy, but it's [probably in the hundreds of millions of dollars](http://kotaku.com/5869042/analyst-pegs-team-fortress-2-hat-economy-at-50-million).


**Who participates in the economy?**

Technically speaking, everyone that is playing the game participates: the game (mostly) distributes the weapons that players can unlock and use for their characters using a time-based drop system, and these items—more specifically the "metal" that they can be "melted down" into—each have a small but non-trivial monetary value.

In reality, the vast majority of trading activities are untaken by a specific subset of "hardcore" players, players who invest incredible amounts of time and money into their activities. Studying these players is not the point of this notebook, but it might be interesting to perform a longitudinal study of trading activity, as I am not aware of one that exists.


**What makes an item valuable?**

The short answer: [Hats](https://www.youtube.com/watch?v=HDI2EmYH98I).

The long answer: Price is primarily a function of two things: rarity and stylishness. Truly valuable items—cosmetics, primarily—drop very rarely. Extremely valuable ones are distributed on promotional or store-only grounds, and do not drop at all. And the absolute most valuable items are "unusual" hats which have various particle effects attached to themselves. Similarly, just as real life fashions can drive prices, so too can Team Fortress 2 ones.


**What is the most valuable item?**

At the intersection of fashion and rarity is the hat to end all hats, the [Burning Flames Unusual Team Captain](https://backpack.tf/stats/Unusual/Team%20Captain/Tradable/Craftable/13). The exact transaction costs of these items is sketchy, but `backpack.tf` says that at one point they were selling for over over 19,000 dollars.

For a virtual hat.

Think about that.


**What is the largest ever single transaction?**

I would love to know myself. The most correct answer might not even be public information.


**What previous efforts have been made to study it?**

None at scale. (a better answer pending further reading)


**How far back does the market data used for this study go?**

`backpack.tf` data goes back to late 2012: the initial log for Earbuds goes back to `2012-08-10`, while the initial log for keys goes back to `2012-07-09`. The constraining variable is Metal, the USD price for which was only first logged on `2013-01-07`. Since all prices are passed through Metal on the way to USD, this means that reliable prices can only be established back to (a reasonably extrapolated) `2013-01-01`.

Future work may extend the augment the timeline back beyond this date, this is up in the air - still exploring alternative data sources.

For the moment price data older than `2013-01-01` is removed, and all prices are normalized to `usd` by default. This could change in the final version.


**Who are you? Why are you doing this?**

In real life I am an undergraduate student of mathematics in New York City. On Steam I go by [ResMar](https://steamcommunity.com/id/residentmario). I invested over 500 hours in Team Fortress 2 when I was younger: I found its economics fascinating and have always wanted to study it in some manner. And I was reading [this book](http://www.amazon.com/Python-Data-Analysis-Wrangling-IPython/dp/1449319793).


**Where can I learn more about this economy?**

My own recommended reading list:

* http://scientificgamer.com/an-economy-of-hats/
* http://www.gamefaqs.com/pc/437678-team-fortress-2/faqs/65012
* http://blogs.valvesoftware.com/economics/arbitrage-and-equilibrium-in-the-team-fortress-2-economy/

I welcome suggestions by others.


In [15]:
from item import Item
from market import Market
import requests

### Market Summary

First we establish some summary statistics about the overall market.

**How many unique items are there?**

An item is considered unique if it has its own texture map in the game files.

In [16]:
requests.get("https://tf2stats.net/all_items/").text.count('<div class="item tooltip round5">')

2192

**How many distinct tradable items are there?**

An item can have one of a number of different "[qualities](https://wiki.teamfortress.com/wiki/Item_quality)". The quality of an item can have a very large effect on the item's value.

Unfortunately asking what the total number of distinct items on the market is non-trivial because of the way that the "unusual" quality items are distributed. We can nevertheless easily get a count of all items of non-unusual quality.

Note that most of the analysis in the remainder of this notebook uses non-unusual items.

**How many distinct non-unusual tradable items are there?**

In [19]:
non_unusual_count = requests.get("http://backpack.tf/pricelist/spreadsheet").text.count("title=")
non_unusual_count

3545

**What is the distribution of the values of non-unusual items?**

Prices are basically determined by the laws of supply (how many are available) and demand (how badly the item is desired). The relative levels of these determine the item's price.

**How strongly does price correlate with numerical availability?**

"Desirability" is a characteristic intrinsic to the players that should be captured by the market price. Let's measure how strongly correlated it is with various metrics.

**How much more than average does the highest-value item in a class of items usually sell for?**

Let's look at the economic history of, say, a hat released in a crate. At first it's value will be precariously high, because it's extremely new, fashionable, and no one has one yet. Within a few days of crate-opening, however, its price will start to plummet, driven down by the large number of people that think as you did, and opened crates and got hats of their own, until it bottoms out a week to two weeks after release. Once the crate is taken off the item drops, however, the hat will slowly start to rebound in value again, until stabalizing at a slowly decaying value once it starts appearing in significant numbers in item drops and crafts.

Now let's talk about bubbles. TF2 naturally feeds bubbles. An item's value is rising, and people take notice. They think "well, if I buy it now and sell it later, I'll make money", so they do that - in bulk. The decreased number on sale and increased demand combine the balloon the price to an artificially high value, up until the point where people realize, wait, what are we doing; then the price falls dramatically: the bubble pops. It happens in the real world, and it happens even more often in the layman-covered TF2 economy (real stock traders avoid bubbles). In the past this has happened to the Max Head (from 4 to 2 buds) and the Genuine Maul (from a bud to 7 to 8 keys), for instance, along with a number of other, generally more minor, items.


In [2]:
earbuds = Item("Earbuds")

In [3]:
earbuds

           currency    value  value_high
2013-01-01      usd   80.860      86.580
2013-01-07      usd   83.720      89.910
2013-01-30      usd   89.910      98.820
2013-02-04      usd   92.880     102.480
2013-02-26      usd   98.820     104.760
2013-03-02      usd   98.020     104.760
2013-03-12      usd  104.760     110.970
2013-03-18      usd  108.000     115.080
2013-03-27      usd  110.970     113.940
2013-03-30      usd  109.720     116.910
2013-04-03      usd  109.720     112.580
2013-04-10      usd  108.250     115.440
2013-04-11      usd  108.250     113.750
2013-04-12      usd  103.920     113.750
2013-04-13      usd  106.560     111.840
2013-04-15      usd  102.120     111.840
2013-04-16      usd  104.650     107.180
2013-04-20      usd  100.100     107.180
2013-04-22      usd  102.520     104.940
2013-04-27      usd  100.170     110.000
2013-05-13      usd  110.000     112.420
2013-05-19      usd  110.000     120.060
2013-05-21      usd  115.000     120.060
2013-05-23      

There are a lot of questions worth asking of this data.

Let's call these our "item classes": unique hats, genuine hats, vintage hats, unusual hats, paint buckets, currencies and psuedo-currencies, action items and taunts, etc...

* How has average item price by category changed over time?
* What does the average hat value curve look like, from the immediate post-release hype to the years that follow?
* Meta: what is the average value percentage adjustment in a `backpack.tf` price?
* How quickly does the value of something from a crate drop?
* etc. etc. etc. etc.

So...first...need to ball all of the API data up into individual CSV files, split up by classes. Since generating a list of "things" in a category is non-trivial to start out let's do some summary statistics on a concrete but interesting subset of the data?

In [4]:
metal = Item("Refined Metal")